In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, BatchNormalization, Activation, Add, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.nn import relu
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.applications import ResNet50

import numpy as np

In [4]:
class MyModel(Model):

    def __init__(self, units=32, activation='relu', **kwargs):
        super().__init__()
        self.hidden1 = Dense(units, activation=activation, name='hidden1')
        self.hidden2 = Dense(units, activation=activation, name='hidden2')
        self.main_out = Dense(1)
        self.aux_out = Dense(1)

    def call(self, inputs):
        in_l, in_r = inputs
        hidden1 = self.hidden1(in_r)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([in_l, hidden2])
        main_out = self.main_out(concat)
        aux_out = self.aux_out(hidden2)
        return main_out, aux_out

model = MyModel()


![img](1.png)

## Implementing ResNet18

![img](2.png)

In [57]:
class IdentityBlock(Model):

    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__()

        self.conv1 = Conv2D(filters, kernel_size, padding='same')
        self.bn1 = BatchNormalization()

        self.conv2 = Conv2D(filters, kernel_size, padding='same')
        self.bn2 = BatchNormalization()

        self.act = Activation('relu')
        self.add = Add()

    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)

        x = self.add([x, input_tensor])
        x = self.act(x)

        return x


class ResNet(Model):

    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = Conv2D(64, 7, padding='same')
        self.bn = BatchNormalization()
        self.act = Activation('relu')
        self.max_pool = MaxPool2D((3, 3))
        self.identity1 = IdentityBlock(64, 3)
        self.identity2 = IdentityBlock(64, 3)
        self.global_pool = GlobalAveragePooling2D()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)

        x = self.identity1(x)
        x = self.identity2(x)

        x = self.global_pool(x)
        return self.classifier(x)

In [82]:
train, test = tf.keras.datasets.mnist.load_data()
train = tf.data.Dataset.from_tensor_slices(train)
train = train.map(lambda x, y: ((tf.cast(x, tf.float32)/255.)[..., np.newaxis], y)).batch(32)
test = tf.data.Dataset.from_tensor_slices(test)
test = test.map(lambda x, y: ((tf.cast(x, tf.float32)/255.)[..., np.newaxis], y)).batch(32)

resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet.fit(train, epochs=1)
resnet.evaluate(test)

313/313 [==============================] - 4s 12ms/step - loss: 0.9323 - accuracy: 0.7785


[0.932255744934082, 0.7785000205039978]